# Prohledávání hyperparametrů pro model BiLSTM nad datasetem TREC (fine) 

Tento notebook slouží k nalezení optimálních hyperparametrů nad datasetem TREC (fine) pro model BiLSTM s odemčenou embedding vrstvou. Hyperparametry jsou hledány pro původní i augmentovaný dataset pro normální trénink i destilaci.

K prohledávání je využito knihovny Optuna s algoritmem Hyperband. Nejlepší konfigurace je volena na základě F1-skóre, zkoušeno je 150 kombinací hyperparametrů pro každou z variant.

## Import knihoven a základní nastavení

In [1]:
from transformers import BasicTokenizer, Trainer
from datasets import concatenate_datasets, load_from_disk
import kagglehub
import optuna
import torch
import math
import base

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


Resetování náhodného seedu pro replikovatelnost výsledků.

In [ ]:
base.reset_seed()

Ověření dostupnosti GPU.

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available and will be used:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("GPU is not available, using CPU.")

GPU is available and will be used: NVIDIA A100 80GB PCIe MIG 2g.20gb


Načtení embeddingů.

Načtení datasetu a jeho základní předzpracování (tokenizace, vytvoření slovníků všech tokenů, vytvoření indexu pro GloVe embeddingy).

In [4]:
my_glove = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
print(my_glove)

/home/jovyan/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1


In [ ]:
GLOVE_FILE = f"{my_glove}/glove.6B.300d.txt"
DATASET = "trec"

In [6]:
train_data = load_from_disk(f"~/data/{DATASET}/train-logits_fine")
eval_data = load_from_disk(f"~/data/{DATASET}/eval-logits_fine")
test_data = load_from_disk(f"~/data/{DATASET}/test-logits_fine")

all_train_data = load_from_disk(f"~/data/{DATASET}/train-logits-augmented_fine")

all_data = concatenate_datasets([load_from_disk(file) for file in [f"~/data/{DATASET}/eval-logits_fine", f"~/data/{DATASET}/test-logits_fine", f"~/data/{DATASET}/train-logits-augmented_fine"]])
tokenizer = BasicTokenizer(do_lower_case=True)

Tokenizace.

In [7]:
train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), train_data))
eval_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), eval_data))
test_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), test_data))

all_train_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_train_data))

all_data_tokens = list(map(lambda e: tokenizer.tokenize(e["sentence"]), all_data))

Získání všech unikátních tokenů v datasetu.

In [8]:
vocab = base.get_vocab(all_data_tokens)

Přiřazení indexu jednotlivým tokenům.

In [ ]:
word_index = dict(zip(vocab, range(len(vocab))))

Získání indexů z GloVe embeddingů.

In [10]:
embeddings_index = base.get_embeddings_indeces(GLOVE_FILE)

Found 400000 word vectors.


Definice velikosti slovníku a velikosti embedding dimenze. 

In [11]:
print(len(vocab))
num_tokens = len(vocab) + 2
embedding_dim = 300

8766


Vytvoření vazby mezi tokeny (jejich indexy) a embeddingy. Část tokenů nebyla nalezena, což ovšem nepředstavuje problém.

In [12]:
embedding_matrix = base.get_embedding_matrix(num_tokens, embedding_dim, word_index, embeddings_index)

Converted 8551 words (215) misses


Přiřazení indexu tokenům v každé části datasetu.

In [13]:
train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),train_data_tokens))
eval_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),eval_data_tokens))
test_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),test_data_tokens))

all_train_data_index = list(map(lambda x: list(map(lambda y: word_index[y], x)),all_train_data_tokens))

Zarovnání délky všech záznamů.

In [14]:
train_padded_data = list(map(lambda x: base.padd(x,60), train_data_index))
eval_padded_data = list(map(lambda x: base.padd(x,60), eval_data_index))
test_padded_data = list(map(lambda x: base.padd(x,60), test_data_index))

all_train_padded_data = list(map(lambda x: base.padd(x,60), all_train_data_index))

Přidání ID tokenů do každé části datasetu.

In [15]:
train_data = train_data.add_column("input_ids", train_padded_data)
eval_data = eval_data.add_column("input_ids", eval_padded_data)
test_data = test_data.add_column("input_ids", test_padded_data)

all_train_data = all_train_data.add_column("input_ids", all_train_padded_data)

Základní konfigurace tréninku během prohledávání. Optuna nepracuje s epochami, ale s kroky. Níže je prováděn přepočet. 

Minimální délka tréninku je pět epoch, maximální 15 epoch. Maximální počet kroků pro warm up je nastaven na 10 % první epochy.

In [16]:
num_epochs = 15
batch_size = 128

In [17]:
data_length = len(train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

## Prohledávání s normálním tréninkem nad původním datasetem
Definice hledaných hyperparametrů a jejich rozmezí.

In [18]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up)
    }   
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [19]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Získání modelu s definovanou odemčenou embedding vrstvou. 

In [20]:
def get_BiLSTM():
    return base.BiLSTMClassifier(embedding_matrix=embedding_matrix, embedding_dim=embedding_dim, fc_dim=400, hidden_dim=300, output_dim=50, freeze_embed=False)

In [21]:
base.reset_seed()

Konfigurace jednotlivých tréninků.

In [22]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

Konfigurace trenéra pro jednotlivé tréninky. 

In [23]:
trainer = Trainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM()
)
  

Nastavení prohledávání.

In [24]:
best_trial_normal = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-embedd",
    n_trials=150
)

[I 2025-03-23 01:13:28,224] A new study created in memory with name: Base-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,3.461800,3.047080,0.189734,0.022144,0.024338,0.013021
2,2.814600,2.617949,0.372136,0.038624,0.078360,0.049747
3,2.443800,2.280283,0.439047,0.068191,0.099587,0.071883
4,2.155800,2.059490,0.483043,0.095430,0.125390,0.100944
5,1.955600,1.895859,0.529789,0.112293,0.150968,0.123581
6,1.763900,1.807152,0.538955,0.176345,0.161470,0.146206
7,1.627800,1.713703,0.568286,0.176519,0.182097,0.166172
8,1.525900,1.628252,0.596700,0.249123,0.203762,0.190628
9,1.415300,1.578906,0.598533,0.230872,0.210060,0.199370
10,1.322700,1.542505,0.613199,0.229849,0.223440,0.214511


[I 2025-03-23 01:14:13,090] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 0}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.916400,2.192074,0.443630,0.104488,0.113677,0.099753
2,1.768000,1.507256,0.628781,0.264674,0.255848,0.241793
3,1.120100,1.210473,0.701192,0.428385,0.362594,0.359856
4,0.657100,1.109886,0.713107,0.473417,0.427416,0.434709
5,0.372800,1.143373,0.752521,0.536336,0.505119,0.504740
6,0.203800,1.221412,0.754354,0.539074,0.541116,0.526224
7,0.097800,1.351081,0.753437,0.601839,0.569406,0.569866
8,0.040600,1.393696,0.759853,0.635720,0.601810,0.600294
9,0.018400,1.511722,0.762603,0.652554,0.598873,0.605533
10,0.011100,1.511352,0.762603,0.634614,0.598893,0.606013


[I 2025-03-23 04:18:14,062] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.004869791072319661, 'weight_decay': 0.003, 'warmup_steps': 1}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.741100,1.989647,0.496792,0.137951,0.144474,0.118082
2,1.488500,1.374242,0.659028,0.344742,0.318835,0.320574
3,0.825100,1.105261,0.747021,0.474300,0.436791,0.441362
4,0.373400,1.124272,0.753437,0.566859,0.532435,0.534080
5,0.142700,1.281250,0.769936,0.669757,0.625789,0.632301
6,0.050300,1.290808,0.787351,0.705799,0.672961,0.674463
7,0.021800,1.336129,0.789184,0.693077,0.645688,0.655009
8,0.006900,1.419090,0.798350,0.727016,0.680836,0.680176
9,0.002000,1.452958,0.798350,0.718131,0.668737,0.676562
10,0.001100,1.480290,0.801100,0.715260,0.674737,0.678505


[I 2025-03-23 04:19:12,171] Trial 149 pruned. 


In [25]:
print(best_trial_normal)

BestRun(run_id='25', objective=0.7213101626581778, hyperparameters={'learning_rate': 0.004865736242064892, 'weight_decay': 0.001, 'warmup_steps': 2}, run_summary=None)


In [26]:
base.reset_seed()

## Prohledávání s destilací nad původním datasetem
Konfigurace jednotlivých tréninků.

In [27]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.


In [28]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [29]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [30]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [31]:
best_trial_distill = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-embedd",
    n_trials=150
)

[I 2025-03-23 04:19:12,662] A new study created in memory with name: Distill-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 3, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,2.239200,2.006961,0.179652,0.024221,0.020668,0.007371
2,1.842500,1.698756,0.380385,0.040339,0.078750,0.050465
3,1.604600,1.493990,0.440880,0.067901,0.100723,0.072117
4,1.434500,1.376686,0.473877,0.104446,0.117615,0.093160
5,1.323400,1.289420,0.509624,0.097179,0.133923,0.107066


[I 2025-03-23 04:19:42,129] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 4, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.749200,1.296109,0.509624,0.119337,0.141604,0.120638
2,1.084700,0.954217,0.653529,0.258881,0.271103,0.256962
3,0.710600,0.748866,0.727773,0.346624,0.368530,0.349308
4,0.441900,0.676705,0.752521,0.486292,0.442425,0.447090
5,0.287000,0.615687,0.779102,0.518675,0.503303,0.499933
6,0.197400,0.603129,0.777269,0.591114,0.553096,0.553294
7,0.144200,0.584630,0.798350,0.668887,0.601725,0.613287
8,0.116400,0.577741,0.793767,0.654347,0.627020,0.626377
9,0.098800,0.559081,0.811182,0.717225,0.655027,0.667206
10,0.089300,0.555744,0.810266,0.713223,0.660072,0.666599


[I 2025-03-23 07:25:23,734] Trial 148 finished with value: 0.6914117762871147 and parameters: {'learning_rate': 0.003911768052522711, 'weight_decay': 0.007, 'warmup_steps': 1, 'lambda_param': 0.8, 'temperature': 7.0}. Best is trial 141 with value: 0.7244345195390912.


Trial 149 with params: {'learning_rate': 0.0030843245155453645, 'weight_decay': 0.001, 'warmup_steps': 0, 'lambda_param': 0.9, 'temperature': 4.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.729600,1.313120,0.501375,0.118281,0.140116,0.117797
2,1.106000,0.979593,0.647113,0.277149,0.264993,0.250147
3,0.748200,0.806822,0.706691,0.337754,0.348813,0.328880
4,0.499200,0.704607,0.732356,0.398933,0.393113,0.382659
5,0.325600,0.633707,0.766269,0.492365,0.463112,0.467973
6,0.229200,0.625932,0.770852,0.529966,0.503541,0.503412
7,0.166600,0.619782,0.780018,0.584847,0.536926,0.545888
8,0.137200,0.612348,0.782768,0.629641,0.566240,0.577316
9,0.112800,0.603116,0.790101,0.655179,0.590251,0.603692
10,0.100500,0.602743,0.785518,0.653416,0.585631,0.599917


[I 2025-03-23 07:26:19,501] Trial 149 pruned. 


In [32]:
print(best_trial_distill)

BestRun(run_id='141', objective=0.7244345195390912, hyperparameters={'learning_rate': 0.004999442141173464, 'weight_decay': 0.008, 'warmup_steps': 0, 'lambda_param': 0.5, 'temperature': 6.5}, run_summary=None)


Přepočet kroků s ohledem na změnu velikosti datasetu.

In [33]:
data_length = len(all_train_data)
min_r = math.ceil(data_length/batch_size)*5
max_r = math.ceil(data_length/batch_size)*num_epochs
warm_up = math.ceil(data_length/batch_size/10)

In [34]:
base.reset_seed()

## Prohledávání s normálním tréninkem nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [35]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-base-embedd-aug_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-base-embedd-aug_fine_hp-search", epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí.

In [36]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [37]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace trenéra pro jednotlivé tréninky. 

In [38]:
trainer = Trainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [39]:
best_trial_normal_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Base-aug-embedd",
    n_trials=150
)

[I 2025-03-23 07:26:19,829] A new study created in memory with name: Base-aug-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 39}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.696900,1.183622,0.701192,0.416820,0.357296,0.360198
2,0.453500,1.065017,0.759853,0.578596,0.528995,0.536193
3,0.146900,1.177877,0.764436,0.605606,0.577759,0.578389
4,0.062000,1.296311,0.773602,0.696767,0.629443,0.644098
5,0.032600,1.440010,0.776352,0.661277,0.621002,0.628949


[I 2025-03-23 07:27:51,676] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.0007875660249889869, 'weight_decay': 0.001, 'warmup_steps': 8}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.588900,1.203988,0.810266,0.765514,0.712079,0.718195
2,0.029400,1.356908,0.806599,0.750163,0.688214,0.700871
3,0.011600,1.384105,0.816682,0.787050,0.747450,0.748715
4,0.007900,1.442833,0.813016,0.795194,0.739312,0.751858
5,0.005000,1.449193,0.816682,0.774763,0.743851,0.746819
6,0.002800,1.518874,0.822181,0.781815,0.755460,0.757788
7,0.003000,1.605371,0.812099,0.775318,0.751073,0.750486
8,0.002000,1.630836,0.819432,0.783770,0.737719,0.750147
9,0.002200,1.590193,0.816682,0.787483,0.739817,0.752582
10,0.000400,1.710855,0.819432,0.785137,0.758758,0.760907


[I 2025-03-23 16:48:23,127] Trial 148 finished with value: 0.7586174858058865 and parameters: {'learning_rate': 0.00224473743989911, 'weight_decay': 0.001, 'warmup_steps': 0}. Best is trial 116 with value: 0.7937875646913016.


Trial 149 with params: {'learning_rate': 0.003937875022867262, 'weight_decay': 0.0, 'warmup_steps': 2}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.482800,1.157240,0.806599,0.772973,0.719817,0.733405
2,0.024700,1.310796,0.805683,0.773378,0.739149,0.740696
3,0.013300,1.542389,0.814849,0.799324,0.753782,0.763041
4,0.008000,1.679979,0.812099,0.782599,0.744096,0.749502
5,0.007200,1.604445,0.813016,0.786253,0.743238,0.750060
6,0.005800,1.780284,0.813932,0.766107,0.756718,0.752112
7,0.003500,1.943498,0.812099,0.808192,0.748132,0.762382
8,0.002200,1.965499,0.822181,0.809607,0.762409,0.776422
9,0.002400,1.989234,0.818515,0.787282,0.770648,0.766751
10,0.000600,2.034948,0.819432,0.775175,0.746415,0.748304


[I 2025-03-23 16:54:01,050] Trial 149 finished with value: 0.7585518183584314 and parameters: {'learning_rate': 0.003937875022867262, 'weight_decay': 0.0, 'warmup_steps': 2}. Best is trial 116 with value: 0.7937875646913016.


In [40]:
print(best_trial_normal_aug)

BestRun(run_id='116', objective=0.7937875646913016, hyperparameters={'learning_rate': 0.003523221778570819, 'weight_decay': 0.003, 'warmup_steps': 6}, run_summary=None)


In [41]:
base.reset_seed()

## Prohledávání s destilací nad augmentovaným datasetem
Konfigurace jednotlivých tréninků.

In [42]:
training_args = base.get_training_args(output_dir=f"~/results/{DATASET}/bilstm-distill-embedd-aug_fine_hp-search", logging_dir=f"~/logs/{DATASET}/bilstm-distill-embedd-aug_fine_hp-search", remove_unused_columns=False, epochs=num_epochs, batch_size=batch_size)

Definice hledaných hyperparametrů a jejich rozmezí, rozšířeno o hyperparametry destilace.

In [43]:
def hp_space(trial):
    params =  {
        "learning_rate": trial.suggest_float("learning_rate", 5e-5, 5e-3, log=True),
        "weight_decay": trial.suggest_float("weight_decay", 0, 1e-2, step=1e-3),
        "warmup_steps" : trial.suggest_int("warmup_steps", 0, warm_up),
        "lambda_param": trial.suggest_float("lambda_param",0,1,step=.1),
        "temperature": trial.suggest_float("temperature", 2,7, step=.5)
    }
    print(f"Trial {trial.number} with params: {params}")
    return params

Konfigurace Optuny.

In [44]:
pruner = optuna.pruners.HyperbandPruner(min_resource=min_r, max_resource=max_r, reduction_factor=2, bootstrap_count=2)
sampler = optuna.samplers.TPESampler(seed=42, multivariate=True)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(


Konfigurace destilačního trenéra pro jednotlivé tréninky. 

In [45]:
trainer = base.DistilTrainer(
    args=training_args,
    train_dataset=all_train_data,
    eval_dataset=eval_data,
    compute_metrics=base.compute_metrics,
    model_init = lambda: get_BiLSTM(),
)
  

Nastavení prohledávání.

In [46]:
best_trial_distill_aug = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    compute_objective=lambda metrics: metrics["eval_f1"],
    pruner=pruner,
    sampler=sampler,
    study_name="Distill-aug-embedd",
    n_trials=150
)

[I 2025-03-23 16:54:01,333] A new study created in memory with name: Distill-aug-embedd


Trial 0 with params: {'learning_rate': 0.0002805758207667253, 'weight_decay': 0.01, 'warmup_steps': 39, 'lambda_param': 0.6000000000000001, 'temperature': 2.5}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,1.160200,0.828101,0.694775,0.314388,0.306172,0.295042
2,0.425100,0.634547,0.780935,0.511661,0.485083,0.485877
3,0.222800,0.585156,0.790101,0.628787,0.552567,0.572560
4,0.148000,0.560218,0.807516,0.718019,0.629717,0.658331
5,0.115000,0.551896,0.808433,0.724440,0.660381,0.681240
6,0.099000,0.552129,0.808433,0.740795,0.665557,0.690026
7,0.090000,0.556047,0.813932,0.743962,0.669970,0.695364
8,0.083800,0.546292,0.811182,0.752920,0.676842,0.703096
9,0.079800,0.546623,0.817599,0.768858,0.703305,0.726095
10,0.076400,0.549268,0.815765,0.770501,0.698507,0.722482


[I 2025-03-23 16:57:43,863] Trial 0 pruned. 


Trial 1 with params: {'learning_rate': 0.00010255552094216992, 'weight_decay': 0.0, 'warmup_steps': 46, 'lambda_param': 0.6000000000000001, 'temperature': 5.5}


...

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.425400,0.524632,0.814849,0.796281,0.726267,0.749085
2,0.087200,0.521625,0.830431,0.810055,0.762792,0.774390
3,0.073100,0.505664,0.833181,0.821729,0.764867,0.781121
4,0.069200,0.509963,0.832264,0.818436,0.774012,0.783384
5,0.066500,0.508577,0.834097,0.839329,0.779575,0.796400
6,0.065400,0.517820,0.829514,0.845525,0.773168,0.795912
7,0.064700,0.516677,0.834097,0.844438,0.782051,0.800258
8,0.063100,0.518138,0.823098,0.804469,0.753054,0.765774
9,0.062400,0.529241,0.823098,0.826259,0.759657,0.777712
10,0.061200,0.513264,0.827681,0.835369,0.767370,0.785918


[I 2025-03-24 03:56:21,053] Trial 148 pruned. 


Trial 149 with params: {'learning_rate': 0.0041337567860579225, 'weight_decay': 0.001, 'warmup_steps': 20, 'lambda_param': 0.9, 'temperature': 6.0}


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.417700,0.499621,0.826764,0.795340,0.737995,0.751829
2,0.085400,0.506764,0.829514,0.838186,0.770784,0.786074
3,0.074000,0.511343,0.823098,0.826300,0.752159,0.769307
4,0.069600,0.523154,0.828598,0.831806,0.759033,0.777205
5,0.066500,0.510450,0.835930,0.844635,0.783174,0.800067
6,0.064600,0.517992,0.829514,0.833363,0.779586,0.793468
7,0.063600,0.521977,0.833181,0.822346,0.780836,0.788704
8,0.062700,0.523429,0.830431,0.837805,0.784446,0.799817
9,0.062300,0.521101,0.828598,0.840038,0.775335,0.795016
10,0.061100,0.522667,0.828598,0.823946,0.763096,0.780421


[I 2025-03-24 03:59:24,354] Trial 149 pruned. 


In [47]:
print(best_trial_distill_aug)

BestRun(run_id='29', objective=0.8166135562260471, hyperparameters={'learning_rate': 0.004465858399905994, 'weight_decay': 0.002, 'warmup_steps': 49, 'lambda_param': 0.9, 'temperature': 2.0}, run_summary=None)
